<a href="https://colab.research.google.com/github/LCaravaggio/felicidad_ivqreg/blob/main/GeoCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!wget "https://ookla-open-data.s3.amazonaws.com/shapefiles/performance/type=fixed/year=2020/quarter=4/2020-10-01_performance_fixed_tiles.zip"

--2023-10-02 18:37:12--  https://ookla-open-data.s3.amazonaws.com/shapefiles/performance/type=fixed/year=2020/quarter=4/2020-10-01_performance_fixed_tiles.zip
Resolving ookla-open-data.s3.amazonaws.com (ookla-open-data.s3.amazonaws.com)... 52.92.176.105, 52.92.148.241, 52.92.192.169, ...
Connecting to ookla-open-data.s3.amazonaws.com (ookla-open-data.s3.amazonaws.com)|52.92.176.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 378399344 (361M) [application/zip]
Saving to: ‘2020-10-01_performance_fixed_tiles.zip’

2020-10-01_performa 100%[===================>] 360.87M  12.1MB/s    in 29s     

2023-10-02 18:37:42 (12.4 MB/s) - ‘2020-10-01_performance_fixed_tiles.zip’ saved [378399344/378399344]



In [7]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [8]:
!pip install simpledbf
from simpledbf import Dbf5
dbf = Dbf5('/content/gps_fixed_tiles.dbf')
df = dbf.to_dataframe()

In [9]:
!pip install pyquadkey2

from pyquadkey2 import quadkey

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 3.8 MB/s eta 0:00:00


In [10]:
def qk_to_geo(quadkey_str, zoom_level):
    qk = quadkey.from_str(quadkey_str)
    lat_lon = qk.to_geo(zoom_level)
    return lat_lon

In [11]:
df['geo'] = df['quadkey'].apply(qk_to_geo , zoom_level=1)

In [12]:
import requests
from bs4 import BeautifulSoup

def Citytolatlon(city):
  api_url = 'http://www.geonames.org/search.html?q={}'
  response = requests.get(api_url + city)
  text=""
  if response.status_code == requests.codes.ok:
    soup = BeautifulSoup(response.text, 'html.parser')
    td_elements = soup.find_all('td', nowrap=True)
    count = 0
    for td in td_elements:
      text += td.get_text(strip=True)
      count+=1

      if count == 2:
          break
  else:
      print("Error:", response.status_code, response.text)
  return text

In [13]:
!pip install latlon3
import latlon

  Preparing metadata (setup.py) ... done
  Created wheel for latlon3: filename=latlon3-1.0.4-py3-none-any.whl size=31367 sha256=c7e3459a87f0a749e6d8fcaad1b9323f29886e7f025fcfd3e3021fe168196713
  Stored in directory: /root/.cache/pip/wheels/80/4d/11/7acb2342b2d16d9253d5cb76bbe83de3b7666bf57e27592ef6
Successfully built latlon3


In [14]:
from latlon import Latitude, Longitude, string2latlon

In [75]:
import re

class LL:
    __name__ = 'LatLon'
    def __init__(self, lat, lon, name=None):

        try:
            if lat.type() == 'GeoCoord':
                self.lat = lat
            else:
                raise AttributeError
        except AttributeError:
            self.lat = Latitude(lat)
        try:
            if lon.type() == 'GeoCoord':
                self.lon = lon
            else:
                raise AttributeError
        except AttributeError:
            self.lon = Longitude(lon)
        self.name = name

def toLatLong (texto):
  pattern = r"([SN])\s+(\d+)°\s+(\d+)[′']\s+(\d+)''([EW])\s+(\d+)°\s+(\d+)[′']\s+(\d+)''"

  matches = re.findall(pattern, texto.replace('�','°'))

  if matches:
    direction1, degrees1, minutes1, seconds1, direction2, degrees2, minutes2, seconds2 = matches[0]
    output_str1 = f"{degrees1} {minutes1} {seconds1} {direction1}"
    output_str2 = f"{degrees2} {minutes2} {seconds2} {direction2}"

  else:
    output_str1="0 0 0 N"
    output_str2="0 0 0 E"

  return string2latlon(output_str1, output_str2, 'd% %m% %S% %H')

In [17]:
import pandas as pd
base=pd.read_stata('LB2020.dta')

In [18]:
lista_ciudades=base['ciudad'].unique()

In [79]:
latsylongs=[]
for ciudad in lista_ciudades:
    # Get the latitude and longitude for the current city
    lat_lon = toLatLong(Citytolatlon(ciudad.encode('latin-1').decode('utf-8')))

    # Append the lat_lon values as a list to the matrix
    latsylongs.append([ciudad, lat_lon.lat, lat_lon.lon])

# Create a DataFrame from the list of lists
ciudades_latlon = pd.DataFrame(latsylongs, columns=['City', 'Latitude', 'Longitude'])

In [104]:
ciudades_latlon['Latitude']=ciudades_latlon['Latitude'].astype(float)
ciudades_latlon['Longitude']=ciudades_latlon['Longitude'].astype(float)

In [105]:
(ciudades_latlon['Latitude'] != 0).sum()

1166

In [106]:
ciudades_latlon['avg_d_kbps']=""
ciudades_latlon['avg_u_kbps']=""
ciudades_latlon['avg_lat_ms']=""

In [116]:
from scipy.spatial import cKDTree
import numpy as np

kdtree = cKDTree(df['geo'].apply(lambda x: (np.radians(x[0]), np.radians(x[1]))).tolist())

In [125]:
def find_nearest_city(row):
    lat, lon = np.radians(row['Latitude']), np.radians(row['Longitude'])
    dist, idx = kdtree.query([lat, lon])
    return pd.Series([df.at[idx, 'avg_d_kbps'], df.at[idx, 'avg_u_kbps'], df.at[idx, 'avg_lat_ms']])

ciudades_latlon[['avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms']] = ciudades_latlon.apply(find_nearest_city, axis=1)

In [126]:
ciudades_latlon

,City,Latitude,Longitude,avg_d_kbps,avg_u_kbps,avg_lat_ms
0,AR: Buenos Aires-Presidente Peron,-34.917778,-58.379167,44754,8949,13
1,AR: Capital Federal-Ciudad de Buenos Aires,-34.613056,-58.377222,58761,13477,14
2,AR: Buenos Aires-Suipacha,-34.750000,-59.750000,6418,1665,49
3,AR: Buenos Aires-Necochea,-38.554444,-58.739444,18383,4087,26
4,AR: TucumÃ¡n-Lules,-26.928611,-65.338333,9471,6714,25
...,...,...,...,...,...,...
1367,VE: Miranda-Acevedo,10.250000,-66.333333,1640,843,150
1368,VE: Monagas-Caripe,10.166667,-63.416667,112,1086,354
1369,VE: Trujillo-Candelaria,9.618056,-70.356944,455,83,202
1370,VE: Trujillo-Carache,9.666667,-70.333333,365,184,104


In [128]:
ciudades_latlon.to_csv('ciudades_latlon.csv')